# ***1. INTRODUCTION***
\section{Introduction}

# ***2. DATA PROCESSING***
\section{Data processing}

# ***3. PRELIMINARIES***
\section{Preliminaries}

Tóm tắt đoạn LaTeX bạn đã cung cấp:

1. **Phát hiện Khuôn Mặt**: Đoạn này không được mô tả chi tiết.

2. **Xác minh Khuôn Mặt**: Mục tiêu chính là xác định liệu hai hình ảnh có thể hiện cùng một người hay không. Phương pháp tiếp cận bao gồm việc sử dụng một hàm mã hóa để tạo ra các vector nhúng từ ảnh, giúp so sánh hiệu quả hơn.

3. **Mã Hóa Khuôn Mặt**: Sử dụng kiến trúc mạng nơ-ron tiên tiến để xử lý hình ảnh khuôn mặt RGB, tạo ra các vector nhúng 128 chiều chứa các đặc trưng nổi bật.

4. **Mất Mát Ba Phần Tử (Triplet Loss)**: Đào tạo mô hình với ba loại hình ảnh: Ảnh Neo (Anchor), Ảnh Dương (Positive) và Ảnh Âm (Negative). Mục tiêu là làm cho mã hóa của Ảnh Neo gần với Ảnh Dương hơn so với Ảnh Âm.

5. **Mất Mát Ba Phần Tử Trực Tuyến (Online Triplet Loss)**: Trong phiên bản này, các bộ ba được chọn động trong mỗi lần lặp đào tạo. Một phương pháp chọn mẫu trong DataLoader được sử dụng để đảm bảo số lượng bộ ba cụ thể trong mỗi lô.

6. **Mất Mát Biên Góc Cộng (Additive Angular Margin Loss)**: Mục tiêu là cải thiện sức mạnh phân biệt của mô hình nhận dạng khuôn mặt bằng cách thêm hình phạt vào góc giữa đặc trưng hiện tại và trọng số mục tiêu.

7. **Tích hợp Mô Hình**: Bao gồm ba bước - Phát hiện và căn chỉnh khuôn mặt, Trích xuất khuôn mặt (Tạo vector nhúng), và Phân loại khuôn mặt (Xác minh khuôn mặt).

Đây là một bản tóm tắt ngắn gọn về các khía cạnh kỹ thuật và lý thuyết liên quan đến xác minh khuôn mặt và các phương pháp liên quan được sử dụng trong xử lý ảnh kỹ thuật số và nhận dạng khuôn mặt.

```latex
\section{Preliminaries}
\subsection{Face Detection}


\subsection{Face Verification}
In Face Verification, the primary objective is to determine whether two images depict the same person. While a simplistic approach involves pixel-wise image comparison, this method proves inefficient due to variations in lighting, facial orientation, and other factors. To address these challenges, an encoding function denoted as $f(\text{img})$, is introduced. This encoding facilitates element-wise comparisons, resulting in more accurate judgments regarding the similarity of two images.

\subsubsection{Face Encoding}
To enhance Face Verification, we utilize advanced neural network architectures to process RGB face images, generating 128-dimensional embedding vectors containing distinctive features of those faces.

Calculating the distance between two embedding vectors and applying a threshold makes it possible to ascertain whether two pictures represent the same person. The effectiveness of an encoding is measured based on the following criteria.

\begin{itemize}
    \item The embeddings of two images of the same person should be notably similar.
    \item The embeddings of two images of different individuals should exhibit significant dissimilarity.
\end{itemize}

\subsubsection{The Triplet Loss}

For an image $x$, denoted as $f(x)$ where $f$ is the function computed by the neural network, training involves triplets of images ($A$, $P$, $N$):

\begin{itemize}
    \item $A$ represents the "Anchor" image – a picture of a person.
    \item $P$ is the "Positive" image – a picture of the same person as the Anchor image.
    \item $N$ is the "Negative" image – a picture of a different person than the Anchor image.
\end{itemize}

These triplets are selected from the training dataset and are represented as ($A^{(i)}$, $P^{(i)}$, $N^{(i)}$) for the $i$-th training example. The objective is to ensure that the encoding of an image $A^{(i)}$ of an individual is closer to the Positive image $P^{(i)}$ than to the Negative image $N^{(i)}$ by at least a margin $\alpha$:
\[
\| f(A^{(i)}) - f(P^{(i)}) \|_2^2 + \alpha \leq \| f(A^{(i)}) - f(N^{(i)}) \|_2^2
\]
This constraint guides the training process to embed similar faces closer in the feature space while pushing dissimilar faces farther apart.

\subsubsection{Online Triplet Loss}
In the original implementation of triplet loss , as known as offline triplet loss, triplets are pre-selected and stored in a dataset before the training starts. Cocretely, we would create batches of these triplets of size \(B\), which means we will have compute \(3B\) embeddings (anchor, positive, negative) to get the \(B\) triplets, compute the loss of these \(B\) triplets and then backpropagate into the network. Within online triplet loss, triplets are selected dynamically during each training iteration. For each training batch, an anchor sample is chosen, and then a positive sample and a negative sample are selected based on the current state of the model. Given a batch of \(B\) examples (for instance \(B\) images of faces), we compute the \(B\) embeddings and we then can find a maximum of $B^{3}$ triplets. In fact, many of these triplets are not valid (i.e. we can not ensure have enough positives and negatives). 

The use of the sampler method in the DataLoader helps ensure a specific number of triplets in each batch. According to the PyTorch Metric Learning library, the MPerClassSampler method allows the specification of the number of samples per class in a batch. For instance, with 4 samples per class and 16 classes per batch, a total of 64 samples per batch can be achieved. This approach enhances stabilization and maximizes the number of triplets during training.
\begin{figure}[h]
    \centering
    \includegraphics[width=\columnwidth]{Image/OnlineLoss/online_triplet.png}
    \caption{Implementation of online triplet.}

    \label{fig:onlinetriplet}
\end{figure}


Method: For each iteration, we have to computed a batch of \(B\) embeddings from a batch of \(B\) inputs. Now our mission is to generate triplets from these embeddings. Whenever we have three elements a,p,n which a and p have the same label but are distinct and n has a different label, so a,p,n is a valid trplet. Then we will build the 3D tensor of shape (\(B\),\(B\),\(B\)) (\(B\) is batch size) where the element at index \((i, j, k)\) represents the loss for triplet\((i, j, k)\), lastly we will set to 0 if the invalid one and take the average over all the valid triplets. 

Additionally, to enhance the efficiency of loss function and challenging examples for the network to learn from, we will select a subset triplet among all of them. Of course, we will not choose random, paper  In Defense of the Triplet Loss for Person Re-Identification introduced 2 strong strategies:

\begin{figure}[h]
    \centering
    \includegraphics[width=\columnwidth]{Image/OnlineLoss/strategies.png}
    \caption{Selection strategies.}

    \label{fig:onlinetriplet}
\end{figure}

\begin{itemize}
    \item •	Hard triplet: for each triplet, negative is closer to anchor than the positve, for example: d(a,p)>d(a,n)
    \item •	Semi-hard triplet: for each triplet, the negative is farther from the anchor than the positive but still provides a challenge for the network. For example:  d(a,p) < d(a,n) < d(a,p) + margin
\end{itemize}

\subsubsection{Additive Angular Margin Loss}

Additive Angular Margin Loss \cite{deng2022arcface} aims to further improve the discriminative power
of the face recognition model and to stabilize the training process by adding a penalty m to the angle between the current feature and the target weight:

\begin{figure}[h]
    \centering
    \includegraphics[width=\columnwidth]{Image/arcface/training.png}
    \caption{Training a DCNN for face recognition supervised by the ArcFace loss \cite{deng2022arcface}. Based on the feature $x_i$ and weight $W$ normalization, we
get the $cos \theta_j$ (logit) for each class as WT
j xi. We calculate the $arccos(\theta_{y_i})$ and get the angle between the feature xi and the ground truth
weight $W_{y_i}$. In fact, $W_j$ provides a kind of centre for each class. Then, we add an angular margin penalty m on the target (ground truth) angle $\theta_{y_i}$. After that, we calculate $cos(\theta_{y_i} + m)$ and multiply all logits by the feature scale $s$. The logits then go through the softmax function and contribute to the cross-entropy loss.}

    \label{fig:arcfacetraining}
\end{figure}

As a result, the full loss function is:
\[L_2 = -\frac{1}{N} \sum_{i = 1}^{N} \frac{e^{s(cos(\theta_{y_i} + m))}}{e^{s(cos(\theta_{y_i} + m))} + \sum_{j = 1, j \neq y_{i}}^{N}e^{s\;cos\theta_j}} \]

By doing so, the proposed ArcFace loss enforce more evident gap between nearest classes comparing to the popular softmax loss' roughly separable feature embedding but noticeably ambiguous decision boundaries.

\begin{figure}[h]
    \centering
    \includegraphics[width=\columnwidth]{Image/arcface/compare_to_softmax.png}
    \caption{Toy examples under the softmax and ArcFace loss on 8 identities with 2D features \cite{deng2022arcface}. Dots indicate samples and lines refer to the centre direction of each identity. Based on the feature normalisation, all face features are pushed to the arc space with a fixed radius. The geodesic distance gap between closest classes becomes evident as the additive angular margin penalty is incorporated.}

    \label{fig:arcfacecompare}
\end{figure}


\subsection{Model Integration}
Face Recognition Problem requires at least the following 3 steps:

\paragraph{Face Detection and Face Alignment}
This step focuses on determining the position of the face in an image or video frame. A rectangle is created around the facial region for precise localization. Additionally, this process includes aligning the facial angle to ensure accuracy and uniformity in identifying facial features.

\paragraph{Face Extraction (Face Embedding)}
After determining the face's position, this step concentrates on extracting facial features. These features are represented as a vector in a high-dimensional space, typically a 128-dimensional vector. This numeric representation of the face is used to differentiate between individuals.

\paragraph{Face Classification (Face Verification)}
The final decision-making step is to classify the face based on the generated feature vector. In this task, the model compares the distance between the feature vector of the new face and those known beforehand in the dataset. A smaller distance indicates similarity between the two faces, allowing the model to determine whether the person in the new image is someone known or unknown from the dataset.


```

# ***5. MODELLING***

## 5.1 Face Detection

### 5.1.1 Yolov5s

\subsection{Face Detection}
\subsubsection{YOLOv5s Model}
In this section, we focus on deploying the YOLOv5s model, specifically tuned for the task of face detection. YOLO (You Only Look Once) is a state-of-the-art, real-time object detection system, and YOLOv8 is the latest in this series. The 's' in YOLOv5s stands for 'small', indicating that this model is the smallest and fastest variant, making it ideal for applications that require real-time processing with limited computational resources.

\paragraph{Model Architecture} The YOLOv5 architectural model is designed for efficiency and speed. It uses the CSPDarknet53 backbone, a modification of the Darknet configuration that was used in previous versions. New configuration of SPPF and CSP-PAN is used for the Neck part. YOLOv5 uses the output of YOLOv3 for the Head section.


1. Backbone: YOLOv5 employs a CSP-Darknet53 backbone, which is an enhanced version of the Darknet53 architecture used in YOLOv3. The CSP (Cross Stage Partial Network) strategy is used here to enable the flow of information to deep layers and reduce redundant gradients.

2. Neck: The neck of the model is designed to construct feature pyramids that aid in detecting objects of various sizes and scales. YOLOv5 incorporates a modified Path Aggregation Network (PANet) with the CSP strategy and a variant of the Spatial Pyramid Pooling (SPP) block called SPPF for faster processing.



3. Head: The head of YOLOv5 is identical to that used in YOLOv3 and YOLOv4. It performs the final prediction, generating bounding box coordinates, objectness scores, and class predictions. 




\paragraph{Input Preprocessing}
For face detection, the input images are first resized to a standard dimension of 640x640 pixels. YOLO labels.txt format (x\_center, y\_center, width, height), normalized by image dimensions.

\paragraph{Detection Mechanism}
YOLOv5s divides the input image into a grid, and each grid cell predicts bounding boxes, objectness scores indicating the likelihood of an object's presence, and class probabilities. For face detection, the model is trained to identify and localize faces within these bounding boxes.

\paragraph{Model Performance}
YOLOv5s demonstrates high accuracy and speed in face detection. Experimentally, we have only trained Yolov5 on the WIDER set with 12880 training images and 3226 validation images, the detection ability of the model with actual images đã quite good. The model's small size also makes it suitable for deployment in mobile applications and embedded systems.

# ***6. RESULTS***

```latex
\section{Result}

\subsection{Evaluation Methods}
\subsubsection{Face Detection}
To evaluate our model's proficiency in face detection, we utilized a set of 1000 images from the CELEBA dataset, which was not included in our training data to ensure unbiased testing. Our assessment focuses on three primary metrics: precision, recall, and mean Intersection over Union (IOU), with a threshold set at 0.5. These metrics are crucial as precision measures the accuracy of the detected faces against the total detected objects, recall evaluates the model's ability to detect all actual faces, and mean IOU assesses the accuracy of the bounding box placement. Additionally, we use Normalized Mean Error (NME) metrics to evaluate model performance in face alignment, NME focuses on the precision of key facial landmarks against ground truth annotations to ensure accurate and reliable alignment. 

\subsubsection{Face Verification}
We evaluated our model using the standardized Labeled Face in the Wild dataset. Concretely, we follow the dataset's instruction to use a 10-fold cross validation using pre-splitted set. We also report the model Validation Rate (True Positive Rate, TPR) with given False Acceptance Rate (False Positive Rate, FAR) of [0.1, 0.01, 0.001] respectively. These two metrics specify how often the model is able to verify an user given a threshold satisfying a pre-defined FAR.

Furthermore, we also collected 60 images of 12 different people and report the TPR given the aforementioned FAR.
```

The description of the YOLOv5s model for face detection in your text is accurate and aligns well with the typical features and applications of the YOLO (You Only Look Once) framework. Let's break down the key points for clarity:

1. **Grid-based Detection**: YOLO models, including YOLOv5s, divide the input image into a grid. Each cell in this grid is responsible for predicting bounding boxes and confidence scores (objectness scores) for objects that the center of the box falls into that cell. This is a standard approach in YOLO models and is known for its efficiency in detecting objects (or faces, in this case).

2. **Objectness Scores and Class Probabilities**: Each grid cell predicts not only the bounding boxes but also objectness scores, which indicate the likelihood of an object's presence in the box. Since YOLOv5s can be used for multiple object detection tasks, it generally also predicts class probabilities. However, in the context of face detection, the model might be specifically trained to recognize and classify faces only.

3. **Model Training on WIDER Face Dataset**: The WIDER Face Dataset is a widely used benchmark for face detection. Training YOLOv5s on this dataset, consisting of a diverse range of faces in various scenarios, would contribute to the robustness of the model in real-world applications.

4. **Performance and Application**: YOLOv5, especially the 's' (small) variant, is known for its balance between speed and accuracy, making it a suitable choice for real-time applications, including mobile and embedded systems. The fact that your experiments show good detection capabilities with actual images further confirms the model's practical effectiveness.

5. **Model Size**: The 's' in YOLOv5s typically indicates a smaller, more efficient version of the model, optimized for speed and size, which makes it particularly suitable for environments where computational resources are limited, like mobile devices or embedded systems.

Overall, your description of YOLOv5s for face detection accurately reflects its mechanism, performance, and typical use cases in the field of computer vision.